### Invocation
- Invoke
- Stream
    - `stream()`, `astream_events()`
- Batch
    - `batch()`, `batch_as_completed()`
- Tool Call
- Structured Output
- Multimodal
- Reasoning
- Caching
- Rate limiting
- Base URL or proxy
- Log probabilities
- Token usage
- Invocation config

### 기본 사용법
채팅 모델
`init_chat_model` 로 초기화

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5-nano")
response = model.invoke("Why do parrots talk?")

### Chat Model Parameters

Chat model의 동작을 설정하는 데 사용되는 파라미터들입니다. 지원되는 파라미터는 모델 및 제공자에 따라 다를 수 있습니다.

**`model`**
-   **Type**: `string`
-   **Required**: Yes
-   **Description**: 사용하려는 특정 모델의 이름이나 식별자입니다.

**`api_key`**
-   **Type**: `string`
-   **Required**: No
-   **Description**: 모델 제공자에 대한 인증에 필요한 키입니다. 보통 모델 접근을 위해 가입할 때 발급되며, 환경 변수로 설정하여 접근하는 경우가 많습니다.

**`temperature`**
-   **Type**: `number`
-   **Required**: No
-   **Description**: 모델 출력의 무작위성을 제어합니다. 값이 높을수록 응답이 더 창의적이 되고, 낮을수록 더 결정론적인 응답이 나옵니다.

**`stop`**
-   **Type**: `string[]`
-   **Required**: No
-   **Description**: 모델이 출력 생성을 중단해야 할 시점을 나타내는 문자 시퀀스입니다.

**`timeout`**
-   **Type**: `number`
-   **Required**: No
-   **Description**: 요청을 취소하기 전, 모델로부터 응답을 기다리는 최대 시간(초)입니다.

**`max_tokens`**
-   **Type**: `number`
-   **Required**: No
-   **Description**: 응답의 총 토큰 수를 제한하여 출력 길이를 효과적으로 제어합니다.

**`max_retries`**
-   **Type**: `number`
-   **Required**: No
-   **Description**: 네트워크 타임아웃이나 속도 제한과 같은 문제로 요청이 실패할 경우, 시스템이 요청을 다시 보내는 최대 시도 횟수입니다.

### Invoke
모델을 호출 하는 가장 간단한 방법

In [ ]:
response = model.invoke("Why do parrots have colorful feathers?")
print(response)

# 대화 내역을 나타내기 위해 메시지 목록을 제공.
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
    SystemMessage("You are a helpful assistant that translates English to French."),
    HumanMessage("Translate: I love programming."),
    AIMessage("J'adore la programmation."),
    HumanMessage("Translate: I love building applications.")
]

response = model.invoke(conversation)
print(response)  # AIMessage("J'adore créer des applications.")

### Stream

In [ ]:
for chunk in model.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

# 모델이 전체 응답을 완료한 후, 단일 객체`AIMessage`를 반환하는 invoke()와 달리, 
# stream()은 출력 테스트의 일부를 포함하는 여러 객체`AIMessageChunk`를 반환.
full = None  # None | AIMessageChunk
for chunk in model.stream("What color is the sky?"):
    full = chunk if full is None else full + chunk
    print(full.text)

# The
# The sky
# The sky is
# The sky is typically
# The sky is typically blue
# ...

print(full.content_blocks)
# [{"type": "text", "text": "The sky is typically blue..."}]

In [ ]:
# astream_events()
# 이벤트 유형 및 기타 메타데이터를 기반으로 필터링이 간소화되고,
# 백그라운드에서 전체 메시지가 집계
async for event in model.astream_events("Hello"):

    if event["event"] == "on_chat_model_start":
        print(f"Input: {event['data']['input']}")

    elif event["event"] == "on_chat_model_stream":
        print(f"Token: {event['data']['chunk'].text}")

    elif event["event"] == "on_chat_model_end":
        print(f"Full message: {event['data']['output'].text}")

    else:
        pass
# Input: Hello
# Token: Hi
# Token:  there
# Token: !
# Token:  How
# Token:  can
# Token:  I
# ...
# Full message: Hi there! How can I help today?

### Batch
모델에 대한 요청을 병렬로 수행

In [ ]:
responses = model.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])
for response in responses:
    print(response)
# 기본적으로 전체 배치에 대한 최종 출력만 반환.
# 생성이 완료될 때마다 각 개별 입력에 대한 출력을 받으려면 
# `batch_as_completed()` 사용
for response in model.batch_as_completed([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]):
    print(response)

# batch(), 또는 batch_as_completed() 사용시 `max_concurrency` 속성으로 최대 병렬 호출 수를 제어
# model.batch(
#     list_of_inputs,
#     config={
#         'max_concurrency': 5,  # Limit to 5 parallel calls
#     }
# )

### Tool Call
`bind_tools()` 이후 호출 시 모델은 필요에 따라 바인딩된 도구를 호출할 수 있음.
- 일반 호출
- 도구 호출 루프
- 도구 호출 강제
- 병렬 도구 호출
- 스트리밍 도구 호출

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

# ---
# * 도구 호출 루프
model_with_tools = model.bind_tools([get_weather])

# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

# ---
# * 도구 호출 강제
model_with_tools = model.bind_tools([tool], tool_choice="any")

# ---
# * 병렬 도구 호출
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke(
    "What's the weather in Boston and Tokyo?"
)

# The model may generate multiple tool calls
print(response.tool_calls)
# [
#   {'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_1'},
#   {'name': 'get_time', 'args': {'location': 'Tokyo'}, 'id': 'call_2'}
# ]


# Execute all tools (can be done in parallel with async)
results = []
for tool_call in response.tool_calls:
    if tool_call['name'] == 'get_weather':
        result = get_weather.invoke(tool_call)
    ...
    results.append(result)

# ---
# * 스트리밍 도구 호출
for chunk in model_with_tools.stream(
    "What's the weather in Boston and Tokyo?"
):
    # Tool call chunks arrive progressively
    if chunk.tool_call_chunks:
        for tool_chunk in chunk.tool_call_chunks:
            print(f"Tool: {tool_chunk.get('name', '')}")
            print(f"Args: {tool_chunk.get('args', '')}")

# Output:
# Tool: get_weather            # Loop 1
# Args:
# Tool:                        # Loop 2
# Args: {"loc
# Tool:                        # Loop 3
# Args: ation": "BOS"}
# Tool: get_time               # Loop 4
# Args:
# Args:
# Tool:                        # Loop 5
# Args: {"timezone": "Tokyo"}

### 구조화된 출력 (Structured Output)
모델은 주어진 스키마와 일치하는 형식으로 응답을 제공하도록 요청할 수 있음. 

In [ ]:
# Pydantic 모델은 필드 검증, 설명 및 중첩 구조를 갖춘 가장 풍부한 기능 세트를 제공
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

# ---
# 구문 분석된 구조와 함께 메시지 출력
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

# 원시 객체 포함: include_raw=True 
model_with_structure = model.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
response
# {
#     "raw": AIMessage(...),
#     "parsed": Movie(title=..., year=..., ...),
#     "parsing_error": None,
# }
# AIMessage와 같은 응답 메타데이터에 엑세스하귀 위해 구문 분석된 표현과 함께 원시 객체를 반환하는 것이 유요할 수도 있음.

# ---
# 중첩 구조
class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)

### Multimodal
특정 모델은 이미지, 오디오, 비디오와 같은 비텍스트 데이터를 처리하고 반환할 수 있음.

In [ ]:
response = model.invoke("Create a picture of a cat")
print(response.content_blocks)
# [
#     {"type": "text", "text": "Here's a picture of a cat"},
#     {"type": "image", "base64": "...", "mime_type": "image/jpeg"},
# ]

### Reasoning
최신 모델은 결론에 도달하기 위해 다단계 추론을 수행할 수 있음. 기본 모델이 뒷받침하는 경우, 이 추론 과정을 표현화하여 모델이 최종 응답에 도달한 과정을 더 잘 이해할 수 있습니다.

In [ ]:
# stream
for chunk in model.stream("Why do parrots have colorful feathers?"):
    reasoning_steps = [r for r in chunk.content_blocks if r["type"] == "reasoning"]
    print(reasoning_steps if reasoning_steps else chunk.text)

# invoke
response = model.invoke("Why do parrots have colorful feathers?")
reasoning_steps = [b for b in response.content_blocks if b["type"] == "reasoning"]
print(" ".join(step["reasoning"] for step in reasoning_steps))

### 캐싱 (Caching)
채팅 모델은 API 호출 속도가 느리고 비용이 많이 들 수 있음. 이를 완화하기 위해 LangChain은 채팅 모델 통합을 위한 캐싱 계층을 제공

In [ ]:
# 기본적으로 캐싱은 비활성화되어 있음. 활성화하려면 다음을 사용.
from langchain_core.globals import set_llm_cache

# ---
# 메모리 캐시
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

response = model.invoke("Tell me a joke")
response = model.invoke("Tell me a joke")  # Fast, from cache

# ---
# SQLite 캐시
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

response = model.invoke("Tell me a joke")
response = model.invoke("Tell me a joke")  # Fast, from cache

### 속도 제한 (Rate limiting)
많은 채팅 모델 제공업체는 특정 기간 동안 수행할 수 있는 호출 횟수에 제한을 둠. 속도 제한에 도달하면 일반적으로 제공업체로부터 속도 제한 오류 응답을 받게 되며, 추가 요청을 하려면 기다려야 함.

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter

# 단위 시간당 요청 수만 제한 가능. 요청 크기에 따라 제한을 적용해야 하는 경우에는 도움이 되지 않음.
rate_limiter = InMemoryRateLimiter(
        requests_per_second=0.1,  # 1 request every 10s
        check_every_n_seconds=0.1,  # Check every 100ms whether allowed to make a request
        max_bucket_size=10,  # Controls the maximum burst size.
)

model = init_chat_model(
    model="gpt-5",
    model_provider="openai",
    rate_limiter=rate_limiter
)

### Base URL or proxy

In [ ]:
# OpenAI
model = init_chat_model(
    model="MODEL_NAME",
    model_provider="openai",
    base_url="BASE_URL",
    api_key="YOUR_API_KEY",
)

# Http Proxy
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o",
    openai_proxy="http://proxy.example.com:8080"
)

### Log probabilities
특정 모델은 주어진 토큰의 발생 가능성을 나타내는 토큰 수준의 로그 확률을 반환하도록 구성할 수 있음.

In [ ]:
model = init_chat_model(
    model="gpt-4o",
    model_provider="openai"
).bind(logprobs=True)

response = model.invoke("Why do parrots talk?")
print(response.response_metadata["logprobs"])

### 토큰 사용 (Token usage)
여러 모델 제공자가 호출 응답의 일부로 토큰 사용 정보를 반환합니다. 이 정보가 제공되는 경우, 해당 모델에서 생성된 AIMessage 객체에 포함

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

llm_1 = init_chat_model(model="openai:gpt-4o-mini")
llm_2 = init_chat_model(model="anthropic:claude-3-5-haiku-latest")

callback = UsageMetadataCallbackHandler()
result_1 = llm_1.invoke("Hello", config={"callbacks": [callback]})
result_2 = llm_2.invoke("Hello", config={"callbacks": [callback]})
callback.usage_metadata
# {
#     'gpt-4o-mini-2024-07-18': {
#         'input_tokens': 8,
#         'output_tokens': 10,
#         'total_tokens': 18,
#         'input_token_details': {'audio': 0, 'cache_read': 0},
#         'output_token_details': {'audio': 0, 'reasoning': 0}},
#         'claude-3-5-haiku-20241022': {'input_tokens': 8,
#         'output_tokens': 21,
#         'total_tokens': 29,
#         'input_token_details': {'cache_read': 0, 'cache_creation': 0}
#     }
# }

### Invocation config

In [ ]:
def my_callback_handler():
    ...
response = model.invoke(
    "Tell me a joke",
        config={
        "run_name": "joke_generation",      # Custom name for this run
        "tags": ["humor", "demo"],          # Tags for categorization
        "metadata": {"user_id": "123"},     # Custom metadata
        "callbacks": [my_callback_handler], # Callback handlers
    }
)

# ---

# configurable_fields 를 지정하여 런타임 구성 가능 모델을 생성할 수도 있음.
from langchain.chat_models import init_chat_model

configurable_model = init_chat_model(temperature=0)

configurable_model.invoke(
    "what's your name",
        config={"configurable": {"model": "gpt-5-nano"}},  # Run with GPT-5-Nano
)
configurable_model.invoke(
    "what's your name",
        config={"configurable": {"model": "claude-3-5-sonnet-latest"}},  # Run with Claude
)